## 1. 설정: 라이브러리 설치

먼저, 이 노트북이 빌드된 특정 버전의 Google Agent Development Kit (ADK)를 설치한다. 버전을 고정하면 코드가 항상 예상대로 작동한다.

## 라이브러리 불러오기

다음으로 Google API 키를 안전하게 제공해야 한다. 이 코드는 키를 붙여넣을 수 있는 보안 입력 프롬프트를 생성한다. 그런 다음 키를 환경 변수로 설정하며, 이는 ADK가 요청을 인증하는 표준 방식이다.

In [48]:
import os
import json
from dotenv import load_dotenv

from google.adk.agents import Agent, SequentialAgent
from google.adk.tools import google_search
from google.adk.tools.agent_tool import AgentTool
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService, Session
from google.genai.types import Content, Part
from IPython.display import display, Markdown

In [49]:
load_dotenv()
MODEL = "gemini-2.5-flash"

## 사용자 정의 도구 정의

여기서는 게스트 관리 에이전트를 위한 사용자 정의 스킬을 정의한다. 이것들은 이전 글에서 게스트 데이터베이스 역할을 하는 딕셔너리와 상호 작용하는 동일한 파이썬 함수들이다.

In [50]:
# 게스트 목록 데이터베이스 역할을 하는 간단한 딕셔너리.
GUEST_DATABASE = {}

def add_guest(name: str, email: str) -> str:
  """게스트의 이름과 이메일을 이벤트 게스트 목록에 추가한다."""
  print(f"도구 실행됨: 이메일 '{email}'인 게스트 '{name}' 추가 중.")
  GUEST_DATABASE[email] = name
  return f"{name}을(를) 게스트 목록에 성공적으로 추가했다."

def get_guest_list() -> str:
  """이벤트에 등록된 모든 게스트의 현재 목록을 가져온다."""
  print("도구 실행됨: 게스트 목록 검색 중.")
  if not GUEST_DATABASE:
    return "게스트 목록이 현재 비어 있다."
  return json.dumps(GUEST_DATABASE)

## 전문가 에이전트 1 생성: 게스트 관리자

이것은 방금 정의한 사용자 정의 도구를 사용하여 이벤트의 게스트 목록을 관리하는 데만 집중하는 첫 번째 전문가 에이전트다.

In [51]:
# 게스트 관리 에이전트 정의
guest_management_agent = Agent(
    name="guest_management_agent",
    model=MODEL,
    description="이벤트의 게스트 목록을 관리하는 전문 에이전트다. 게스트를 추가하고 현재 목록을 검색할 수 있다.",
    instruction="""
    당신은 게스트 관리 어시스턴트다. 당신의 유일한 업무는 게스트를 목록에 추가하거나 요청 시 전체 목록을 가져오는 것이다.

    - add_guest 도구를 사용하여 새 사람을 목록에 추가한다.
    - get_guest_list 도구를 사용하여 현재 게스트 목록을 표시한다.

    이벤트 기획이나 정보 검색과 같은 다른 작업은 수행하지 않는다.
    도구를 사용하여 게스트 목록을 관리하는 것에만 엄격히 집중한다.
    """,
    tools=[add_guest, get_guest_list]
)

## 전문가 에이전트 2 생성: 이벤트 기획자

이것은 Google 검색을 사용하여 이벤트를 기획하는 전문가인 두 번째 전문가다. 여기서 주요 변경 사항은 `output_key="event_plan"`을 추가하는 것인데, 이는 ADK가 이 에이전트의 출력을 `event_plan`이라는 상태 변수에 저장하도록 지시한다.

In [52]:
def create_event_planner_agent():
    """이벤트 기획자 에이전트 생성"""
    return Agent(
        name="event_planner_agent",
        model="gemini-2.5-flash",
        description="테마, 장소, 일정을 검색하여 이벤트를 기획하는 전문가 에이전트다. 사용자 피드백을 바탕으로 계획을 수정할 수 있다.",
        instruction="""
        당신은 전문 이벤트 기획자다. 당신의 유일한 목표는 이벤트를 위한 포괄적인 계획을 세우는 것이다.

        프로세스는 다음과 같다:
        1.  **테마 브레인스토밍:** 2-3가지 창의적인 테마를 제안한다.
        2.  **장소 찾기:** 검색 도구를 사용하여 3-5곳의 잠재적 장소를 찾는다.
        3.  **의제 작성:** 이벤트에 대한 높은 수준의 일정을 제안한다.
        4.  **명확하게 요약:** 최종 계획을 체계적인 형식으로 제시한다.

        게스트 목록이나 다른 작업은 처리하지 않는다. 기획에만 집중한다.
        최종 출력은 구조화된 계획 세부 정보만 포함해야 한다.
        """,
        tools=[google_search],
        # 이 에이전트의 출력을 'event_plan'이라는 키로 저장하여 다음 단계에서 사용할 수 있게 한다.
        output_key="event_plan"
    )

event_planner_agent = create_event_planner_agent()
print(f"에이전트 '{event_planner_agent.name}'가 생성되어 준비되었다!")

에이전트 'event_planner_agent'가 생성되어 준비되었다!


## 전문가 에이전트 3 생성: 커뮤니케이터

이것은 이 글을 위한 새로운 전문가다. 유일한 업무는 공지 이메일 초안을 작성하는 것이다. 지침은 `event_planner_agent`가 제공할 `{event_plan}`이라는 입력 상태 변수를 찾도록 지시한다.

In [54]:
def create_communications_agent():
    """공지 사항 초안 작성을 담당하는 에이전트를 생성한다."""
    return Agent(
        name="communications_agent",
        model=MODEL,
        description="확정된 이벤트 계획을 바탕으로 공지 이메일 초안을 작성하는 전문가 에이전트다.",
        instruction="""
        당신은 전문 커뮤니케이션 어시스턴트다. 당신의 유일한 임무는 제공된 이벤트 계획 {event_plan}의 세부 정보를 가져와 참석자를 위한 명확하고 간결하며 흥미로운 공지 이메일을 작성하는 것이다.
        당신이 받는 입력은 이벤트 계획의 전체 텍스트일 것이다.
        유일한 임무는 해당 이벤트 계획을 참석자를 위한 명확하고 간결하며 흥미로운 공지 이메일로 변환하는 것이다.
        계획을 묻지 말고, 주어진 입력을 사용하라.
        이메일에는 다음 주요 세부 정보가 포함되어야 한다:
        - 이벤트 제목
        - 날짜 및 시간
        - 장소 주소
        - 테마나 의제에 대한 간략한 언급
        """,
        # 이 에이전트는 외부 도구를 사용하지 않는다.
        tools=[]
    )
event_communications_agent = create_communications_agent()
print(f"에이전트 '{event_communications_agent.name}'가 생성되어 준비되었다!")

에이전트 'communications_agent'가 생성되어 준비되었다!


## 순차적 워크플로 구축

여기서는 SequentialAgent를 사용하여 다단계 워크플로를 정의한다. `event_planner_agent`와 `event_communications_agent`를 함께 연결한다. ADK는 자동으로 순서대로 실행하고 첫 번째 에이전트의 `event_plan` 출력을 두 번째 에이전트의 입력으로 전달한다.

In [55]:
# 순차적 에이전트 정의: 기획 -> 통신
event_announcement_agent = SequentialAgent(
    name="event_announcement_agent",
    sub_agents=[event_planner_agent, event_communications_agent],
    description="먼저 이벤트를 기획한 다음 공지 사항을 보내는 워크플로다.",
)

## 전체 에이전트 팀 구성

이제 오케스트레이터를 업그레이드한다. AgentTool을 사용하여 새로운 `event_announcement_agent` 순차적 워크플로를 포함한 전문가들을 래핑한다. `root_orchestrator_agent`는 이제 간단한 작업이나 복잡한 다단계 워크플로를 팀에 위임할 수 있다.

In [56]:
# 에이전트들을 도구로 래핑하여 오케스트레이터가 사용할 수 있게 한다.
guest_list_manager_tool = AgentTool(agent=guest_management_agent)
event_announcement_tool = AgentTool(agent=event_announcement_agent)

In [57]:
root_orchestrator_agent = Agent(
    name="root_orchestrator_agent",
    model=MODEL,
    description="이벤트 기획 및 게스트 관리를 위해 전문 에이전트에게 작업을 위임하는 메인 오케스트레이터다.",
    instruction="""
    당신은 진행 중인 대화를 관리하는 루트 오케스트레이터다. 당신의 임무는 사용자 요청의 전체 맥락을 이해하고 올바른 전문 도구에 위임하는 것이다.

    - 사용자가 이벤트를 기획하거나, 장소를 찾거나, 일정을 만들고 이메일 공지를 보내려면 `event_announcement_tool`을 사용한다.
    - 사용자가 게스트를 추가, 조회 또는 관리하려면 `guest_list_manager_tool`을 사용한다.

    사용자의 질문에 직접 대답하려고 하지 않는다. 항상 적절한 도구에 위임한다. 도구의 응답을 요약하지 않는다.

    """,
    tools=[event_announcement_tool, guest_list_manager_tool]
)

## 실행 엔진 구축

이것은 쿼리를 실행하기 위한 헬퍼 함수이며, 이전 글과 변경되지 않았다. Runner를 초기화하고 `run_async`로 이벤트를 스트리밍하는 핵심 ADK 로직을 처리한다.

In [58]:
async def run_agent_query(agent: Agent, query: str, session: Session, user_id: str):
    """Runner를 초기화하고 주어진 에이전트 및 세션에 대해 쿼리를 실행한다."""
    print(f"\n에이전트 쿼리 실행 중: '{agent.name}', 세션: '{session.id}'...")

    runner = Runner(
        agent=agent,
        session_service=session_service,
        app_name=agent.name
    )

    final_response = ""
    try:
        async for event in runner.run_async(
            user_id=user_id,
            session_id=session.id,
            new_message=Content(parts=[Part(text=query)], role="user")
        ):
            if event.is_final_response():
                final_response = event.content.parts[0].text
    except Exception as e:
        final_response = f"오류 발생: {e}"


    print("\n" + "-"*50)
    print("최종 응답:")
    display(Markdown(final_response))
    print("-"*50 + "\n")

    return final_response

## 세션 서비스 초기화 및 워크플로 실행

마지막으로 `InMemorySessionService`를 설정하고 메인 실행 블록을 정의한다. 이 코드는 오케스트레이터를 위한 단일 세션을 생성한 다음, 간단한 게스트 관리자와 복잡한 순차적 공지 워크플로 모두로 라우팅할 수 있는 방법을 보여주기 위해 일련의 쿼리를 보낸다.

In [59]:
# --- 세션 서비스 초기화 ---
# 이 하나의 서비스가 이 노트북의 모든 다른 세션을 관리한다.
session_service = InMemorySessionService()
user_id = "adk_event_planner_001"

In [60]:
async def run_stateful_orchestrator():

  # 세션 생성
  session = await session_service.create_session(
        app_name=root_orchestrator_agent.name,
        user_id=user_id
  )

  # 첫 번째 쿼리: 이벤트 기획 요청 (순차적 에이전트 실행)
  query1 = "샌프란시스코에서 30명을 위한 소규모 기술 모임을 기획해줘."
  print(f"사용자: {query1}\n")
  await run_agent_query(root_orchestrator_agent, query1, session, user_id)

  # 두 번째 쿼리: 게스트 추가 요청 (게스트 관리 에이전트 실행)
  query2 = "게스트 목록에 이메일 'tim.a@example.com'인 'Tim Apple'을 추가해줘."
  print(f"사용자: {query2}\n")
  await run_agent_query(root_orchestrator_agent, query2, session, user_id)

  # 세 번째 쿼리: 게스트 목록 확인 요청 (게스트 관리 에이전트 실행)
  query3 = "이제 게스트 목록을 보여줘."
  print(f"사용자: {query3}\n")
  await run_agent_query(root_orchestrator_agent, query3, session, user_id)

# 전체 시스템 실행
await run_stateful_orchestrator()

사용자: 샌프란시스코에서 30명을 위한 소규모 기술 모임을 기획해줘.


에이전트 쿼리 실행 중: 'root_orchestrator_agent', 세션: 'a827642a-4eb0-4330-b76e-2af6bbf52662'...



--------------------------------------------------
최종 응답:


샌프란시스코 소규모 기술 모임이 기획되었습니다. 다음은 세부 정보입니다:

**제목:** 샌프란시스코 소규모 기술 모임에 초대합니다: 혁신을 위한 연결의 밤!

**날짜 및 시간:**
*   **날짜:** 추후 공지 예정
*   **시간:** 저녁 6:00 PM – 9:00 PM

**장소 주소:**
*   샌프란시스코 내 엄선된 코워킹/이벤트 공간 (정확한 주소는 곧 공개됩니다!)
    *   (WeWork 535 Mission St, Spaces, Presidio Event Space 등 유수의 장소들이 고려되고 있습니다.)

**이번 모임에서 다룰 내용:**
이번 모임은 "AI & 미래 기술", "개발자 생산성 해킹", "샌프란시스코 스타트업 생태계 조명" 등 다양한 최신 기술 트렌드 중 하나를 집중적으로 다룰 예정입니다. 예를 들어, **'AI & 미래 기술'** 테마에서는 인공지능의 최신 동향과 산업 전반에 미치는 영향에 대한 전문가 발표와 심층적인 논의가 이루어질 것입니다.

**주요 의제:**
*   **6:00 PM - 6:30 PM:** 환영 및 등록, 초기 네트워킹
*   **6:30 PM - 6:40 PM:** 개회 및 테마 소개
*   **6:40 PM - 7:40 PM:** 키노트/라이트닝 토크 세션 (선택된 테마 기반)
*   **7:40 PM - 8:10 PM:** 패널 토론 / Q&A 세션
*   **8:10 PM - 8:55 PM:** 자유 네트워킹 및 다과 (음료 및 핑거푸드 제공)
*   **8:55 PM - 9:00 PM:** 폐회

최대 30명의 기술 전문가들이 모여 최신 지식을 공유하고, 영감을 얻으며, 네트워킹을 통해 시너지를 창출할 수 있는 뜻깊은 자리가 될 것입니다.

정확한 날짜와 장소는 확정되는 즉시 다시 공지해 드리겠습니다.

--------------------------------------------------

사용자: 게스트 목록에 이메일 'tim.a@example.com'인 'Tim Apple'을 추가해줘.


에이전트 쿼리 실행 중: 'root_orchestrator_agent', 세션: 'a827642a-4eb0-4330-b76e-2af6bbf52662'...
도구 실행됨: 이메일 'tim.a@example.com'인 게스트 'Tim Apple' 추가 중.

--------------------------------------------------
최종 응답:


Tim Apple 님을 게스트 목록에 추가했습니다.

--------------------------------------------------

사용자: 이제 게스트 목록을 보여줘.


에이전트 쿼리 실행 중: 'root_orchestrator_agent', 세션: 'a827642a-4eb0-4330-b76e-2af6bbf52662'...
도구 실행됨: 게스트 목록 검색 중.

--------------------------------------------------
최종 응답:


현재 게스트 목록은 다음과 같습니다: {"tim.a@example.com": "Tim Apple"}

--------------------------------------------------

